In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:16pt;}
div.text_cell_render.rendered_html{font-size:16pt;}
div.output {font-size:12pt; 
las; font-size:16pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
table.dataframe{font-size:16px;}
</style>
"""))

# <span style="color:red">ch9. vectorEmbedding</span>
# 문장 > 임베딩 벡터(1차원 숫자 배열)
- openAI API의 키(text-embedding-3-large)를 .env에 추가
- upstage(https://console.upstage.ai) 의 키 UPSTAGE_API_KEY를 .env에 추가
# 1. 환경변수 load

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')
upstage_key = os.getenv('UPSTAGE_API_KEY')
print(openai_key[:2])
print(upstage_key[:2])

sk
up


# 2. 유사도 계산하는 
- 1. 유클리드 거리 : 두 벡터간 거리가 가까운지
- 2. 코사인 유사도 : 두 벡터간 방향이 유사한지
- 3. dot product : 두 벡터간 곱을 사용하여 거리와 방향을 모두 고려

In [3]:
import numpy as np
def cosine_similarity(vec1, vec2):
    """두 백터 사이의 코사인 유사도 계산"""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1) 
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1==0 or norm_vec2==0:
        return 0.0
    return dot_product / (norm_vec1*norm_vec2)

# 3. openAI API의 embedding model 사용
- text-embedding-3-large

In [4]:
from openai import OpenAI
openai_client = OpenAI(
    # api_key=openai_key
)

In [5]:
# text-embedding-3-large(과금)
response = openai_client.embeddings.create(
    input= "The king is the prince's father",
    model="text-embedding-3-large"
)

In [6]:
response.usage

Usage(prompt_tokens=7, total_tokens=7)

In [7]:
king_vector = np.array(response.data[0].embedding) # The king is ~ 문장을 숫자로 바꾼 vec
print(king_vector.shape)
print(king_vector)

(3072,)
[ 0.03364377  0.03378009 -0.00439973 ...  0.01654925  0.01048982
 -0.00589925]


In [8]:
queen_response = openai_client.embeddings.create(
    input="The queen is the prince's mother",
    model="text-embedding-3-large"
)

In [9]:
queen_response.usage

Usage(prompt_tokens=7, total_tokens=7)

In [10]:
queen_vector = np.array(queen_response.data[0].embedding)
print(queen_vector.shape)
print(queen_vector)

(3072,)
[ 0.01908419  0.00117847 -0.00398764 ...  0.00576628  0.00822409
  0.01082648]


In [11]:
# king_vector와 queen_vector의 유사도
king_queen_similarity = cosine_similarity(king_vector, queen_vector)
print("king문장과 queen문장의 유사도 :", king_queen_similarity)

king문장과 queen문장의 유사도 : 0.6953682106144796


In [13]:
slave_response = openai_client.embeddings.create(
    input = "The slave begs",
    model="text-embedding-3-large"
)

In [14]:
slave_vector = np.array(slave_response.data[0].embedding)
print(slave_vector.shape)
print(slave_vector)

(3072,)
[-0.00868506  0.00686     0.00754387 ... -0.00823628 -0.0112752
  0.00194153]


In [15]:
king_slave_similarity = cosine_similarity(king_vector, slave_vector)
print("king문장과 slave문장 유사도 :", king_slave_similarity)

king문장과 slave문장 유사도 : 0.18329285510786322


In [ ]:
# 한국어 문장을 벡터로 바꿔도 유사도가 비슷

In [16]:
response = openai_client.embeddings.create(
    input="왕은 왕자의 아버지다",
    model="text-embedding-3-large"
)

In [17]:
kor_king_vector = np.array(response.data[0].embedding)
print(kor_king_vector.shape)
print(kor_king_vector)

(3072,)
[0.01460524 0.0081415  0.00299669 ... 0.00721534 0.01149217 0.00849063]


In [18]:
response = openai_client.embeddings.create(
    input="여왕은 왕자의 어머니다",
    model="text-embedding-3-large"
)

In [19]:
kor_queen_vector = np.array(response.data[0].embedding)
print(kor_queen_vector)

[-0.00633463  0.01295298  0.00534164 ... -0.00638844 -0.00050108
  0.0337521 ]


In [20]:
print("왕문장과 여왕문장의 유사도 :", cosine_similarity(kor_king_vector, kor_queen_vector))

왕문장과 여왕문장의 유사도 : 0.5744286874279486


# 4. upstage의 embedding model사용
- 한국어에서는 openai보다 성능이 좋다

In [23]:
upstage_client = OpenAI(
    api_key=upstage_key,
    base_url="https://api.upstage.ai/v1"
)

In [24]:
response = upstage_client.embeddings.create(
    input="The king is prince's father",
    model="embedding-query"
)

In [25]:
up_king_vector = np.array(response.data[0].embedding)
print(up_king_vector.shape)
print(up_king_vector)

(4096,)
[ 6.99263182e-05 -3.04292757e-02 -2.85108201e-03 ... -7.22163310e-03
  1.04733631e-02  3.70790288e-02]


In [26]:
response = upstage_client.embeddings.create(
    input="The queen is prince's mother",
    model="embedding-query"
)
up_queen_vector = np.array(response.data[0].embedding)

In [27]:
print("king문장과 queen문장의 유사도 :", cosine_similarity(up_king_vector, up_queen_vector))

king문장과 queen문장의 유사도 : 0.6695324547493329


In [28]:
response = upstage_client.embeddings.create(
    input="왕은 왕자의 아버지다",
    model="embedding-query"
)

In [29]:
up_kor_king_vector = np.array(response.data[0].embedding)
print(up_kor_king_vector.shape)

(4096,)


In [30]:
response = upstage_client.embeddings.create(
    input="여왕은 왕자의 어머니다",
    model="embedding-query"
)
up_kor_queen_vector = np.array(response.data[0].embedding)
print(up_kor_queen_vector)

[-0.01259613 -0.00148392 -0.0004282  ... -0.00082159 -0.00722885
  0.0259552 ]


In [31]:
print("왕문장과 여왕문장의 유사도 :", cosine_similarity(up_kor_king_vector,up_kor_queen_vector))

왕문장과 여왕문장의 유사도 : 0.6502462939208556


In [32]:
print("king문장과 왕문장의 유사도 :", cosine_similarity(up_king_vector,
                                             up_kor_king_vector))

king문장과 왕문장의 유사도 : 0.849464981641128
